In [1]:
import deepchem as dc
import pandas as pd
import numpy as np
from rdkit import Chem

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,KFold

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (D:\anaconda\envs\PI3K\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
data = pd.read_csv('../../data/refined_gabaa.csv')

# Feature extraction & Data splitting

In [3]:
featurizer = dc.feat.CircularFingerprint(size=1024,radius=4)
X = featurizer.featurize(data['smiles'])
Y = data['label']
dataset = dc.data.DiskDataset.from_numpy(X=X,y=Y,ids=data['smiles'])

In [6]:
splitter = dc.splits.ScaffoldSplitter()
train_dataset,test_dataset = splitter.train_test_split(dataset=dataset,frac_train=0.7,seed=200)

# GridSearchCV

In [7]:
scoring = {
          'F1':make_scorer(f1_score),
          'AUC':make_scorer(roc_auc_score),
           'BA':make_scorer(balanced_accuracy_score),
           'MCC':make_scorer(matthews_corrcoef)
}

rf_param_grid = {'n_estimators':[150,200,250,300,350,400,450,500],
              'max_depth':[20,30,40,50,60,70,80,90,100,110],
            'max_features':["auto", "sqrt", "log2"]}
    
    
rf_classifier = RandomForestClassifier()
cv = KFold(n_splits=5, shuffle=True, random_state=200)

rf_gs = GridSearchCV(
                    rf_classifier,
                   rf_param_grid,
                 scoring = scoring,
                  cv = cv,
                 n_jobs = -1,
               refit = 'F1',
           return_train_score = True)

rf_gs_fit = rf_gs.fit(train_dataset.X, train_dataset.y.ravel())

In [8]:
val_F1 = rf_gs.best_score_
val_AUC = rf_gs.cv_results_['mean_test_AUC'][rf_gs.best_index_]
val_BA = rf_gs.cv_results_['mean_test_BA'][rf_gs.best_index_]
val_MCC = rf_gs.cv_results_['mean_test_MCC'][rf_gs.best_index_]

# GridSearchCV_Result

In [9]:
print('Best parameters: ', rf_gs.best_params_)
print('Best score (F1): ', rf_gs.best_score_)
print('AUC: ', rf_gs.cv_results_['mean_test_AUC'][rf_gs.best_index_])
print('BA: ', rf_gs.cv_results_['mean_test_BA'][rf_gs.best_index_])
print('MCC: ',rf_gs.cv_results_['mean_test_MCC'][rf_gs.best_index_])

Best parameters:  {'max_depth': 110, 'max_features': 'auto', 'n_estimators': 350}
Best score (F1):  0.8197026816875053
AUC:  0.8609386902224678
BA:  0.8609386902224678
MCC:  0.74160665082966


In [10]:
rf_model = rf_gs_fit.best_estimator_
rf_model

RandomForestClassifier(max_depth=110, n_estimators=350)

# Evaluate model

In [11]:
y_test_pred_proba = rf_model.predict_proba(test_dataset.X)

In [12]:
y_test_pred = rf_model.predict(test_dataset.X)

In [13]:
rf_test_pred_list = []
for test_score in y_test_pred_proba:
    test_score = test_score[1]
    rf_test_pred_list.append(test_score)

In [14]:
rf_test_pred_array = np.array(rf_test_pred_list)

In [15]:
confusion_matrix(test_dataset.y, y_test_pred)

array([[156,  22],
       [155, 240]], dtype=int64)

In [16]:
tn, fp, fn, tp = confusion_matrix(test_dataset.y, y_test_pred).ravel()

In [17]:
print('TN:', tn)
print('FP:', fp)
print('FN:', fn)
print('TP:', tp)

TN: 156
FP: 22
FN: 155
TP: 240


In [18]:
test_F1 = f1_score(test_dataset.y,np.round(rf_test_pred_array))
test_AUC = roc_auc_score(test_dataset.y,rf_test_pred_list)
test_BA =  balanced_accuracy_score(test_dataset.y,np.round(rf_test_pred_array))
test_MCC = matthews_corrcoef(test_dataset.y,np.round(rf_test_pred_array))

# Finall result

In [19]:
rf_performance_dataset = {
    'F1':[val_F1,test_F1],
    'AUC':[val_AUC,test_AUC],
    'BA':[val_BA,test_BA],
    'MCC':[val_MCC,test_MCC],
}

In [21]:
rf_performance = pd.DataFrame(rf_performance_dataset,index=['val','test'])
rf_performance

,F1,AUC,BA,MCC
val,0.819703,0.860939,0.860939,0.741607
test,0.730594,0.866313,0.742000,0.449596
